<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 5:  Web Scraping
### Finding Underpriced RVs on Craigslist

![](https://snag.gy/WrdUMx.jpg)

In this project we will be practicing our web scraping skills.  You can use Scrapy or Python requests in order to complete this project.  It may be helpful to write some prototype code in this notebook to test your assumptions, then move it into a Python file that can be run from the command line.

> In order to run code from the command line, instead of the notebook, you just need to save your code to a file (with a .py extension), and run it using the Python interpreter:<br><br>
> `python my_file.py`

You will be building a process to scrape a single category of search results on Craigslist, that can easily be applied to other categories by changing the search terms.  The main goal is to be able to target and scrape a single page given a set of parameters.

**If you use Scrapy, provide your code in a folder.**

## Import your libraries for scrapy / requests / pandas / numpy / etc
Setup whichever libraries you need. Review past material for reference.

In [1]:
# PREPARE REQUIRED LIBRARIES

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1.  Scrape for the largest US cities (non-exhaustive list)
Search, research, and scrape Wikipedia for a list of the largest US cities.  There are a few sources but find one that is in a nice table.  We don't want all cities, just signifficant cities.  Examine your source.  Look for what can be differentiable.

- Use requests
- Build XPath query(ies)
- Extract to a list
- Clean your list

In [ ]:
# import the necessities
from bs4 import BeautifulSoup as bs4
import urllib2
import numpy as np
import requests
import pandas as pd
import craigslist 

In [8]:
# find the table of top cities sorted by population
wiki = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
page = urllib2.urlopen(wiki)
soup = bs4(page)
table = soup.find("table", { "class" : "wikitable sortable" })

In [4]:
# designate where information should go

rank = []
city = []
state = []
est_2015 = []
census_2010 = []

for row in table.findAll("tr"):
    cells = row.findAll("td")
    if len(cells) > 0:
        rank.append(cells[0].find(text=True))
        city.append(cells[1].find(text=True))
        state.append(cells[2].find(text=True))
        est_2015.append(cells[3].find(text=True))
        census_2010.append(cells[4].find(text=True))
        #print rank, city, state, est_2015, census_2010
        
d = {"city": city, "state": state, "estimated_15": est_2015, 
          "census_2010": census_2010}

df = pd.DataFrame(d, index = rank, columns = ["city", "state", "estimated_15", "census_2010"])
df.head(20)

,city,state,estimated_15,census_2010
1,New York,New York,"8,550,405","8,175,133"
2,Los Angeles,California,"3,971,883","3,792,621"
3,Chicago,Illinois,"2,720,546","2,695,598"
4,Houston,Texas,"2,296,224","2,100,263"
5,Philadelphia,Pennsylvania,"1,567,442","1,526,006"
6,Phoenix,Arizona,"1,563,025","1,445,632"
7,San Antonio,Texas,"1,469,845","1,327,407"
8,San Diego,California,"1,394,928","1,307,402"
9,Dallas,Texas,"1,300,092","1,197,816"
10,San Jose,California,"1,026,908","945,942"


<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1.2 Only retain cities with properly formed ASCII

Optionally, filter out any cities with impropper ASCII characters.  A smaller list will be easier to look at.  However you may not need to filter these if you spend more time scraping a more concise city list.  This list should help you narrow down the list of regional Craigslist sites.

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2.  Write a function to capture current pricing information via Craigslist in one city.
Choose a city from your scraped data, then go to the cooresponding city section on Craigslist, searching for "rv" in the auto section.  Write a method that pulls out the prices.

In [10]:
# san francisco craigslist rv search
url = 'http://sfbay.craigslist.org/search/sss?sort=rel&query=rv'

In [18]:
def find_prices(url):
    rsp = requests.get(url)
    html = bs4(rsp.text, 'html.parser')
    prices = html.find_all('p', attrs={'class': 'row'})
    titles = html.find_all('a', attrs={"class": "hdrlnk"})
    dates = html.find_all("time")
    results = []
    for price, title, date in zip(prices, titles, dates):
        price = price.find('span', {'class': 'price'})
        if price is not None:
            price = float(price.text.strip('$'))
        else:
            price = np.nan
        
        title_ = title.get_text()
        date_ = date.get('title')
        results.append([price, title_, date_])
    return results


def next_page(url, url_base = 'http://sfbay.craigslist.org'):
    rsp = requests.get(url)
    text = bs4(rsp.text)
    #print text
    next_page_rel = text.find('a', {'title': 'next page'}).get('href')
    next_page = url_base + next_page_rel
    return next_page 

In [19]:
results = find_prices('http://sfbay.craigslist.org/search/sss?sort=rel&query=rv')

[[33016.0,
  u'2016 Keystone RV Premier 30RIPR***1/2 TON TOWABLE***LARGE LIVING SPACE***',
  u'Fri 21 Oct 02:43:26 PM']]

In [30]:
def find_titles(url):
    #url_base = 'http://sfbay.craigslist.org/search/sss?sort=rel&query=rv'
    rsp = requests.get(url)
    html = bs4(rsp.text, "html.parser")
    results = html.find_all('a', attrs={"class": "hdrlnk"})
    #print results
    titles = []
    for title in results:
        title_ = title.get_text()
        titles.append(title_)
    return titles

In [31]:
def find_dates(url):
    rsp = requests.get(url)
    html = bs4(rsp.text, "html.parser")
    results = html.find_all("time")
    #rsp = requests.get(url)
    #print results
    dates = []
    for date in results:
        date_ = date.get_text()
        dates.append(date_)
    return dates

In [32]:
url = 'http://sfbay.craigslist.org/search/sss?sort=rel&query=rv'
prices = {}
for i in range(10):
    prices[i] = find_prices(url)
    url = next_page(url)
for l in prices.values():
    print len(l)

100
100
100
100
100
100
100
100
100
100


In [33]:
url = 'http://sfbay.craigslist.org/search/sss?sort=rel&query=rv'
titles = {}
for i in range(10):
    titles[i] = find_titles(url)
    url = next_page(url)
#print titles
for l in titles.values():
    print len(l)

100
100
100
100
100
100
100
100
100
100


In [34]:
url = 'http://sfbay.craigslist.org/search/sss?sort=rel&query=rv'
dates = {}
for i in range(10):
    dates[i] = find_dates(url)
    url = next_page(url)
#print titles
for l in dates.values():
    print len(l)

100
100
100
100
100
100
100
100
100
100


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2.1 Create a mapping of cities to cooresponding regional Craigslist URLs

Major US cities on Craigslist typically have their own cooresponding section (ie: SFBay Area, NYC, Boston, Miami, Seattle, etc).  Later, you will use these to query search results for various metropolitian regions listed on Craigslist.  Between the major metropolitan Craigslist sites, the only thing that will differ is the URL's that correspond to them.

The point of the "mapping":  Create a data structure that allows you to iterate with both the name of the city from Wikipedia, with the cooresponding variable that that will allow you to construct each craigslist URL for each region.

> For San Francsico (the Bay Area metropolitan area), the url for the RV search result is:
> http://sfbay.craigslist.org/search/sss?query=rv
>
> The convention is http://[region].craigslist.org/search/sss?query=rf
> Replacing [region] with the cooresponding city name will allow you to quickly iterate through each regional Craigslist site, and scrape the prices from the search results.  Keep this in mind while you build this "mapping".


In [36]:
city_dict = {"los angeles": "https://losangeles.craigslist.org/search/sss?query=rv&sort=rel",
          "san diego": "https://sandiego.craigslist.org/search/sss?query=rv&sort=rel",
          "san jose": "https://sfbay.craigslist.org/search/sby/sss?query=rv&sort=rel",
          "san francisco": "https://sfbay.craigslist.org/search/sss?sort=rel&query=rv"
          }

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. Define a function to caculate mean and median price per city.

Now that you've created a list of cities you want to scrape, adapt your solution for grabbing data in one region site, to grab data for all regional sites that you collected, then calculate the mean and median price of RV results from each city.

> Look at the URLs from a few different regions (ie: portland, phoenix, sfbay), and find what they have in common.  Determine the area in the URL string that needs to change the least, and figure out how to replace only that portion of the URL in order to iterate through each city.

In [42]:
def grab_prices(city_name):
    url = city_dict[city_name]
    for i in range(5):
        prices[i] = find_prices(url)
        url = next_page(url)
    flat_price_list = [p for price_page in prices.values() for p in price_page]
    return flat_price_list
#grab_prices("los angeles")

In [43]:
def grab_titles(city_name):
    url = city_dict[city_name]
    for i in range(5):
        titles[i] = find_titles(url)
        url = next_page(url)
    flat_title_list = [t for title_page in titles.values() for t in title_page]
    return flat_title_list
#grab_titles("los angeles")

In [44]:
def grab_dates(city_name):
    url = city_dict[city_name]
    for i in range(5):
        dates[i] = find_dates(url)
        url = next_page(url)
    flat_dates_list = [d for date_page in dates.values() for d in date_page]
    return flat_dates_list
#grab_dates("los angeles")

<img src="http://imgur.com/l5NasQj.png" style="float: left;margin: 25px 15px 0px 0px; height: 25px">

## 5. Do an analysis of the RV market.

Go head we'll wait.  Anything notable about the data?

In [45]:
columns = ["City", "Price"]
my_dict = {
 "City": ["SF"] * len(grab_prices("san francisco")) + 
         ["LA"] * len(grab_prices("los angeles")) +
         ["SD"] * len(grab_prices("san diego")) +
         ["SJ"] * len(grab_prices("san jose")),
 "Prices": grab_prices("san francisco") +
           grab_prices("los angeles") +
           grab_prices("san diego") +
           grab_prices("san jose"),
 "Title":  grab_titles("san francisco") +
           grab_titles("los angeles") +
           grab_titles("san diego") +
           grab_titles("san jose"),
 "Date":   grab_dates("san francisco") +
           grab_dates("los angeles") +
           grab_dates("san diego") +
           grab_dates("san jose")
}

df = pd.DataFrame(my_dict)


In [46]:
df.head()

,City,Date,Prices,Title
0,SF,Oct 20,13995.0,2013 Cruiser RV Shadow Cruiser S-185FBS (4345)
1,SF,Oct 20,29995.0,2011 Keystone RV Avalanche 290RL (4359)
2,SF,Oct 20,42995.0,2016 Keystone RV Outback 315FRE (10397)
3,SF,Oct 20,23495.0,2016 Keystone RV Outback Terrain 245TBH - Stoc...
4,SF,Oct 20,17999.0,SALE - 2016 Keystone RV Springdale 18ft. with ...


In [47]:
df.shape

(3891, 4)

In [48]:
df.groupby("City").mean()
# it's better to sell in LA and buy in SJ

,Prices
City,
LA,25354.787645
SD,23877.729219
SF,24882.052830
SJ,22724.629827


In [49]:
df.groupby("City").median()

,Prices
City,
LA,14999.0
SD,12000.0
SF,14588.0
SJ,10300.0


In [50]:
df.groupby(["Date", "City"]).mean()

Prices
Date   City              
Oct  1 SJ     2000.000000
Oct  3 LA     4500.000000
       SD     4500.000000
       SF     4500.000000
       SJ     7018.823529
Oct  5 LA      250.000000
       SD      250.000000
       SF      250.000000
       SJ      250.000000
Oct  6 LA    26900.000000
       SD    26900.000000
       SF    26900.000000
       SJ    26900.000000
Oct  7 SJ    14900.000000
Oct 10 SJ    19800.000000
Oct 12 LA     9500.000000
       SD     9500.000000
       SF     9500.000000
       SJ     9500.000000
Oct 13 LA    17516.391304
       SD    17861.931818
       SF    17861.931818
       SJ    23051.377778
Oct 14 LA    16100.250000
       SD    15360.952381
       SF    15571.666667
       SJ     8029.135135
Oct 15 LA    11532.181818
       SD    11889.906250
       SF    11987.416667
...                   ...
Oct 20 SD    24409.657588
       SF    25482.265152
       SJ    26036.559387
Sep 20 SJ     3500.000000
Sep 21 LA      170.000000
       SD      170.000000
       SF      170.000000
       SJ      170.000000
Sep 22 LA      170.000000
       SD      170.000000
       SF      170.000000
       SJ      170.000000
Sep 23 LA      140.000000
       SD      140.000000
       SF      140.000000
       SJ      140.000000
Sep 25 LA     4545.000000
       SD     4545.000000
       SF     4545.000000
       SJ     4545.000000
Sep 27 SJ      250.000000
Sep 28 LA    16500.000000
       SD    16500.000000
       SF    16500.000000
       SJ    16500.000000
Sep 29 SJ    20900.000000
Sep 30 LA    11997.500000
       SD    11997.500000
       SF    11997.500000
       SJ    26211.000000

[78 rows x 1 columns]

In [51]:
df["Month"] = df["Date"].apply(lambda x: x.split(" ")[0])

In [52]:
df.head()

,City,Date,Prices,Title,Month
0,SF,Oct 20,13995.0,2013 Cruiser RV Shadow Cruiser S-185FBS (4345),Oct
1,SF,Oct 20,29995.0,2011 Keystone RV Avalanche 290RL (4359),Oct
2,SF,Oct 20,42995.0,2016 Keystone RV Outback 315FRE (10397),Oct
3,SF,Oct 20,23495.0,2016 Keystone RV Outback Terrain 245TBH - Stoc...,Oct
4,SF,Oct 20,17999.0,SALE - 2016 Keystone RV Springdale 18ft. with ...,Oct


In [53]:
df.groupby("Month").mean()

,Prices
Month,
Oct,24425.706016
Sep,9475.476190


In [54]:
df.groupby("Month").count()
# September is "low" because the scraping settings only go so far back

,City,Date,Prices,Title
Month,,,,
Oct,3849,3849,3075,3849
Sep,42,42,42,42


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.1 Does it makes sense to buy RVs in one region and sell them in another?

Assuming the cost of shipping or driving from one regional market to another.

In [ ]:
# see above

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.2 Can you pull out the "make" from the markup and include that in your analyis?
How reliable is this data and does it make sense?

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.3 Are there any other variables you could pull out of the markup to help describe your dataset?

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 6. Move your project into scrapy (if you haven't used Scrapy yet)

>Start a project by using the command `scrapy startproject [projectname]`
> - Update your settings.py (review our past example)
> - Update your items.py
> - Create a spiders file in your `[project_name]/[project_name]/spiders` directory

You can update your spider class with the complete list of craigslist "start urls" to effectively scrape all of the regions.  Start with one to test.

Updating your parse method with the method you chose should require minimal changes.  It will require you to update your parse method to use the response parameter, and an item model (defined in items.py).

In [ ]:
# see attached .py's

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 7.  Chose another area of Craigslist to scrape.

**Choose an area having more than a single page of results, then scrape multiple regions, multiple pages of search results and or details pages.**

This is the true exercise of being able to understand how to succesffuly plan, develop, and employ a broader scraping strategy.  Even though this seems like a challenging task, a few tweeks of your current code can make this very managable if you've pieced together all the touch points.  If you are still confused as to some of the milestones within this process, this is an excellent opportunity to round out your understanding, or help you build a list of questions to fill in your gaps.

_Use Scrapy!  Provide your code in this project directory when you submit this project._

## Optional: Interview Questions

---- 

### SQL Practice

1)
We have a deliveries table with 3000 rows

`SELECT * FROM deliveries; 
-- 3000 rows in set (0.05 sec) `

15 of those orders are from a customer with the customer_id_number=32

`SELECT * FROM deliveries WHERE customer_id_number = 32;
-- 15 rows in set (0.10 sec)`

Yet, when we SELECT the number of orders that are not from customer_id_number = 32, we only get 2960 results:

`SELECT * FROM deliveries WHERE customer_id_number <> 32;
-- 2960 rows in set (0.11 sec)`

**Question: What’s wrong? And why might this be the case? Modify your code to fix this. **

2) Construct the following tables:

`mysql> SELECT * FROM Employee;
+--------+----------+--------+
| emp_id | emp_name | salary |
+--------+----------+--------+
| 1      | James    |   2000 |
| 2      | Jack     |   4000 |
| 3      | Henry    |   6000 |
| 4      | Tom      |   8000 |
+--------+----------+--------+
4 rows IN SET (0.00 sec)`


`mysql> SELECT * FROM Department;
+---------+-----------+
| dept_id | dept_name |
+---------+-----------+
| 101     | Sales     |
| 102     | Marketing |
| 103     | Finance   |
| 104     | Accounting   |
+---------+-----------+
3 rows IN SET (0.00 sec)`


`mysql> SELECT * FROM Register;
+--------+---------+
| emp_id | dept_id |
+--------+---------+
|      1 |     101 |
|      2 |     102 |
|      3 |     103 |
|      4 |     102 |
+--------+---------+
4 rows IN SET (0.00 sec)`

** Questions: ** 
- Which employees belong to which department? Show this using one line of code (hint: more than one join) 
- What is the total marketing salary? 
- Using a join, can you show that there are no employees in accounting? 



3) Given an Employee table which has 3 fields – Id (Primary key), Salary and Manager Id, where manager id is the id of the employee that manages the current employee, find all employees that make more than their manager in terms of salary. Create the table and write the code that finds this


--- 
### Predictive Modeling

- What are some differences you would expect in a regression model that minimizes squared error, versus a model that minimizes absolute error? In which cases would each error  metric be appropriate?

- What error metric would you use to evaluate how good a binary classifier is? What if the classes are imbalanced?  What if there are more than 2 groups?

- What are various ways to predict a binary response variable? Can you compare two of them and tell me when one would be more appropriate? What’s the difference logistic regression and SVMs? 

- What is the difference between the loss functions used by SVMs and Logistic Regression? 

- What is R-squared? What are some other metrics that could be better than R-squared and why?

- You run your regression on different subsets of your data, and find that in each subset, the beta value for a certain variable varies wildly. What could be the issue here?


--- 
### Coding Questions 

- Given a sorted array and a number x, find a pair in array whose sum is closest to x. What is the time complexity of your algorithm? 
    
    `Examples:`
        Input: arr[] = {10, 22, 28, 29, 30, 40}, x = 54
        Output: 22 and 30

        Input: arr[] = {1, 3, 4, 7, 10}, x = 15
        Output: 4 and 10
        
- Check out this video on Linear Time Algorithm for finding the median: https://www.youtube.com/watch?v=_xntajCBLoE. Implement your version of this algorithm in Python. 

- Search in an almost sorted array: Given an array which is sorted, but after sorting some elements are moved to either of the adjacent positions, i.e., arr[i] may be present at arr[i+1] or arr[i-1]. Write an efficient function to search an element in this array. Basically the element arr[i] can only be swapped with either arr[i+1] or arr[i-1]. For example consider the array {2, 3, 10, 4, 40}, 4 is moved to next position and 10 is moved to previous position. [Hint: You can do this O(log n) time complexity]

    `Examples: `
        Input: arr[] =  {10, 3, 40, 20, 50, 80, 70}, key = 40
        Output: 2 
        Output is index of 40 in given array

        Input: arr[] =  {10, 3, 40, 20, 50, 80, 70}, key = 90
        Output: -1
        -1 is returned to indicate element is not present

In [ ]:
#1 coding question 1
import numpy as np
def find_best_pair(arr, val):
    L, R = 0, len(arr) -1
    diff = float("inf")
    while L < R:
        temp_sum = arr[L] + arr[R]
        temp_diff = abs(temp_sum - val)
        if temp_diff < diff:
            diff = temp_diff
            close_pair = arr[L], arr[R]
        if temp_sum > val:
            R -= 1
        elif temp_sum == val:
            return close_pair
        else:
            L += 1
    return close_pair
find_best_pair([1, 2, 5, 9, 11, 17, 56], 24)
            